In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [27]:
class InceptionModule(keras.layers.Layer):
    def __init__(self, filters, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get('relu')
        
        self.conv_a    = keras.layers.Conv2D(filters[0], kernel_size=(1,1), padding='same', use_bias='false', strides=1)
        self.bn_a      = keras.layers.BatchNormalization()
        
        self.conv_b1   = keras.layers.Conv2D(filters[1], kernel_size=(1, 1), padding='same', use_bias='false', strides=1)
        self.conv_b2   = keras.layers.Conv2D(filters[2], kernel_size=(3, 4), padding='same', use_bias='false', strides=1)
        self.bn_b      = keras.layers.BatchNormalization()
        
        self.conv_c1   = keras.layers.Conv2D(filters[3], kernel_size=(1, 1), padding='same', use_bias='false', strides=1)
        self.conv_c2   = keras.layers.Conv2D(filters[4], kernel_size=(5, 6), padding='same', use_bias='false', strides=1)
        self.bn_c      = keras.layers.BatchNormalization()
        
        self.maxpool_d = keras.layers.MaxPooling2D(pool_size=(3, 4), padding='same', strides=1)
        self.conv_d    = keras.layers.Conv2D(filters[5], kernel_size=(1, 1), padding='same', use_bias='false', strides=1)
        self.bn_d      = keras.layers.BatchNormalization()
        
    def call(self, x):
        out1 = self.conv_a(x)
        out1 = self.bn_a(out1)
        out1 = self.activation(out1)
        
        out2 = self.conv_b1(x)
        out2 = self.conv_b2(out2)
        out2 = self.bn_b(out2)
        out2 = self.activation(out2)
        
        out3 = self.conv_c1(x)
        out3 = self.conv_c2(out3)
        out3 = self.bn_c(out3)
        out3 = self.activation(out3)
        
        out4 = self.maxpool_d(x)
        out4 = self.conv_d(out4)
        out4 = self.bn_d(out4)
        out4 = self.activation(out4)
        
        return tf.concat([out1, out2, out3, out4], axis=3)

In [28]:
class LocalResponseNormalization(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(**kwargs).__init__()
    
    def call(self, x):
        return tf.nn.local_response_normalization(x)

In [29]:
GoogLeNet = keras.Sequential()

GoogLeNet.add(keras.layers.Conv2D(filters=64, kernel_size=(5, 8), strides=2, padding='same', input_shape=(240, 320, 1)))
GoogLeNet.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
GoogLeNet.add(LocalResponseNormalization())
GoogLeNet.add(keras.layers.Conv2D(filters=64, kernel_size=(1, 1), strides=1, padding='same'))
GoogLeNet.add(keras.layers.Conv2D(filters=192, kernel_size=(3, 4), strides=1, padding='same'))
GoogLeNet.add(LocalResponseNormalization())
GoogLeNet.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
GoogLeNet.add(InceptionModule([64, 96, 128, 16, 32, 32]))
GoogLeNet.add(InceptionModule([128, 128, 192, 32, 96, 64]))
GoogLeNet.add(LocalResponseNormalization())
GoogLeNet.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
GoogLeNet.add(InceptionModule([192, 96, 208, 16, 48, 64]))
GoogLeNet.add(InceptionModule([160, 112, 224, 24, 64, 64]))
GoogLeNet.add(InceptionModule([128, 128, 256, 24, 64, 64]))
GoogLeNet.add(InceptionModule([112, 144, 288, 32, 64, 64]))
GoogLeNet.add(InceptionModule([256, 160, 320, 32, 128, 128]))
GoogLeNet.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
GoogLeNet.add(InceptionModule([64, 96, 128, 16, 32, 32]))
GoogLeNet.add(InceptionModule([64, 96, 128, 16, 32, 32]))
GoogLeNet.add(keras.layers.GlobalAveragePooling2D())
GoogLeNet.add(keras.layers.Dropout(0.4))
GoogLeNet.add(keras.layers.Dense(1024, activation='relu'))
GoogLeNet.add(keras.layers.Dense(4, activation='softmax'))

In [30]:
GoogLeNet.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_33 (Conv2D)          (None, 120, 160, 64)      2624      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 60, 80, 64)       0         
 g2D)                                                            
                                                                 
 local_response_normalizatio  (None, 60, 80, 64)       0         
 n_12 (LocalResponseNormaliz                                     
 ation)                                                          
                                                                 
 conv2d_34 (Conv2D)          (None, 60, 80, 64)        4160      
                                                                 
 conv2d_35 (Conv2D)          (None, 60, 80, 192)       147648    
                                                      